In [32]:
#Solution adapted from Kris C Naik
#GitHub https://github.com/krishnaik06/Stock-Price-Prediction-using-Keras-and-Recurrent-Neural-Networ/blob/master/rnn.py
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import autocorrelation_plot


In [33]:
import pandas_datareader as dr
from datetime import datetime
import pandas_datareader.data as web

In [35]:
start=datetime(2015,1,1)
end=datetime(2019,4,1)

In [45]:
# tesla = web.DataReader('TSLA', 'google', start,end)
microsoft = dr.data.get_data_yahoo('MSFT', start=start, end=end)

dataset = microsoft

training_set = dataset.iloc[:, 3:4].values
print(dataset.head())
print(training_set)

                 High        Low       Open      Close      Volume  Adj Close
Date                                                                         
2015-01-02  47.419998  46.540001  46.660000  46.759998  27913900.0  42.418739
2015-01-05  46.730000  46.250000  46.369999  46.330002  39673900.0  42.028660
2015-01-06  46.750000  45.540001  46.380001  45.650002  36447900.0  41.411785
2015-01-07  46.459999  45.490002  45.980000  46.230000  29114100.0  41.937946
2015-01-08  47.750000  46.720001  46.750000  47.590000  29645200.0  43.171677
[[ 46.75999832]
 [ 46.33000183]
 [ 45.65000153]
 ...
 [117.94000244]
 [119.01999664]
 [119.19000244]]


In [47]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [48]:
# Creating a data structure with 63 (approx. 1 quarter) timesteps and 1 output (next day value)
X_train = []
y_train = []
for i in range(63, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-63:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(X_train.shape[0])
print(len(y_train))

63
1006


In [50]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
#https://keras.io/layer/recurrent
from keras.models import Sequential
from keras.layers import Dense #output layer
from keras.layers import LSTM
from keras.layers import Dropout #for each Epox deactivate those that result in 0

In [51]:
# Initialising the RNN
regressor = Sequential()

In [52]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [53]:
# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 30, batch_size = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
1006/1006 [==============================] - 18s 18ms/step - loss: 0.1212
Epoch 2/30
1006/1006 [==============================] - 6s 6ms/step - loss: 0.0218
Epoch 3/30
1006/1006 [==============================] - 6s 6ms/step - loss: 0.0093
Epoch 4/30
1006/1006 [==============================] - 6s 6ms/step - loss: 0.0052
Epoch 5/30
1006/1006 [==============================] - 6s 6ms/step - loss: 0.0042
Epoch 6/30
1006/1006 [==============================] - 5s 4ms/step - loss: 0.0041
Epoch 7/30
1006/1006 [==============================] - 5s 5ms/step - loss: 0.0035
Epoch 8/30
1006/1006 [==============================] - 7s 7ms/step - loss: 0.0041
Epoch 9/30
1006/1006 [==============================] - 7s 7ms/step - loss: 0.0038
Epoch 10/30
1006/1006 [==============================] - 6s 6ms/step - loss: 0.0036
Epoch 11/30
1006/1006 [==============================] - 6s 6ms/step - loss: 0.0036
Epoch 12/30
1006/1006 [============

In [69]:
# Setting Real stock price
real_stock_price = test_data

# Getting the predicted stock price
dataset_total = dataset['Close']
inputs = dataset_total[len(dataset_total) - len(test_data) - 63:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(63, 380):
    X_test.append(inputs[i-63:i, 0])
X_test = np.array(X_test)


X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualising the results
plt.plot(1,1)
plt.plot(real_stock_price, color = 'red', label = 'Real Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

TypeError: 'tuple' object is not callable

In [55]:
from pandas.tseries.offsets import DateOffset

In [56]:
future_dates = [dataset.index[-1] + DateOffset(days=x) for x in range(1,365)]

In [57]:
x = np.array(future_dates.index)

future_dates['forecast'] = regressor.predict(np.array(future_dates.index))

ValueError: Error when checking input: expected lstm_2_input to have 3 dimensions, but got array with shape ()

Model, fit and plot data using regressor calculated above.

In [25]:
# Prolog - Auto Generated #
import os, matplotlib.pyplot, uuid, pandas

# dataset.sort_values(by=['date'], inplace=True, ascending=True)
# dataset['date'] = pandas.to_datetime(dataset['date'])
# dataset.set_index("date", inplace = True)

model_dataset = dataset.iloc[:, 2:3].values 

# dataset = pandas.DataFrame(date, Close, Gain, ema, Volume, sma_10, sma_20, sma_50, sma_100, rsi)
# dataset = dataset.drop_duplicates()
print(model_dataset)
print(len(model_dataset))

[[ 46.65999985]
 [ 46.36999893]
 [ 46.38000107]
 ...
 [118.06999969]
 [118.94999695]
 [119.05999756]]
1069


In [26]:
 # First calculate the mid prices from the highest and lowest

train_data = model_dataset[:750]
test_data = model_dataset[750:]

print(train_data.transpose())
print(len(test_data))

[[46.65999985 46.36999893 46.38000107 45.97999954 46.75       47.61000061
  47.41999817 46.97000122 45.95999908 46.22000122 45.31000137 46.29999924
  45.93999863 46.38000107 47.36000061 47.         42.95000076 42.74000168
  40.93000031 41.54999924 40.59000015 41.63000107 41.93999863 42.22000122
  42.68000031 42.24000168 42.74000168 42.65000153 42.65999985 43.38000107
  43.97000122 43.63000107 43.18000031 43.50999832 43.70000076 44.15000153
  43.95000076 43.99000168 44.13000107 43.66999817 43.56000137 43.00999832
  43.06999969 43.         42.18999863 42.34999847 42.31000137 41.33000183
  40.70000076 41.47000122 41.36999893 41.43000031 42.25999832 42.56000137
  42.88000107 42.77999878 42.91999817 41.22000122 41.11999893 41.09999847
  40.77999878 40.59999847 40.65999985 40.34000015 41.61000061 41.45999908
  41.25       41.63000107 41.40000153 41.79999924 41.75999832 41.95000076
  41.66999817 41.72999954 43.         42.66999817 42.88999939 45.65999985
  47.22999954 47.77999878 48.72000122 

In [27]:
# Scale the data to be between 0 and 1
# When scaling remember! You normalize both test and train data with respect to training data
# Because you are not supposed to have access to test data
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(train_data)